**TODO:**

* Implement data export.
* Implement table sorting.
* Do a better job updating gaps when we insert / delete cells.
* Table-aware plot embedding.
    * It should be possible to embed a bar plot or a line plot simply by assigning a special "formatter" to a row / column.
    * The layout of the embedded plot should be responsive to cell spacing, gaps, padding, etc.
    * The tricky part will be handling formatters that get overridden / are discontiguous.

In [1]:
import numpy
import toyplot.data

import logging
toyplot.log.setLevel(logging.DEBUG)

In [2]:
canvas, table = toyplot.table(trows=3, rows=3, brows=3, lcolumns=3, columns=3, rcolumns=3)
table.cells.cells.data = numpy.arange(81).reshape(9, 9)
table.cells.grid.hlines[...] = "single"
table.cells.grid.vlines[...] = "single"
table.top.left.cells.style = {"fill":"red", "opacity":0.3}
table.top.cells.style = {"fill":"orange", "opacity":0.3}
table.top.right.cells.style = {"fill":"yellow", "opacity":0.3}
table.right.cells.style = {"fill":"greenyellow", "opacity":0.3}
table.bottom.right.cells.style = {"fill":"green", "opacity":0.3}
table.bottom.cells.style = {"fill":"aqua", "opacity":0.3}
table.bottom.left.cells.style = {"fill":"blue", "opacity":0.3}
table.left.cells.style = {"fill":"purple", "opacity":0.3}

#table.cells.gaps.rows[2::3] = 10

#table.cells.row.insert(after=8)
table.cells.cell[2:4,2:4].merge()
table._cell_data

array([[0, 1, 2, 3, 4, 5, 6, 7, 8],
       [9, 10, 11, 12, 13, 14, 15, 16, 17],
       [18, 19, 20, 20, 22, 23, 24, 25, 26],
       [27, 28, 20, 20, 31, 32, 33, 34, 35],
       [36, 37, 38, 39, 40, 41, 42, 43, 44],
       [45, 46, 47, 48, 49, 50, 51, 52, 53],
       [54, 55, 56, 57, 58, 59, 60, 61, 62],
       [63, 64, 65, 66, 67, 68, 69, 70, 71],
       [72, 73, 74, 75, 76, 77, 78, 79, 80]], dtype=object)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 22 23 24 25 26 27 28 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 20